In [ ]:
import pandas as pd

# Load the datasets
fraud_data = pd.read_csv('D:/week8&9 data/Fraud_Data.csv')
ip_to_country = pd.read_csv('D:/week8&9 data/IpAddress_to_Country.csv')
credit_card = pd.read_csv('D:/week8&9 data/creditcard.csv')

In [3]:
# Display column names of fraud_data
print(fraud_data.columns)


Index(['user_id', 'signup_time', 'purchase_time', 'purchase_value',
       'device_id', 'source', 'browser', 'sex', 'age', 'ip_address', 'class'],
      dtype='object')


Data Preparation

Feature and Target Separation

In [22]:
# Feature and Target Separation
X_creditcard = credit_card.drop('Class', axis=1)
y_creditcard = credit_card['Class']

X_fraud = fraud_data.drop('class', axis=1)
y_fraud = fraud_data['class']


In [23]:
from sklearn.preprocessing import LabelEncoder

# Identify categorical columns in fraud_data
categorical_columns = ['device_id', 'source', 'browser', 'sex']

# Apply label encoding to categorical columns
label_encoders = {}
for col in categorical_columns:
    encoder = LabelEncoder()
    fraud_data[col] = encoder.fit_transform(fraud_data[col])
    label_encoders[col] = encoder

# Check the transformed data
print(fraud_data.head())


   user_id          signup_time        purchase_time  purchase_value  \
0    22058  2015-02-24 22:55:49  2015-04-18 02:47:11              34   
1   333320  2015-06-07 20:39:50  2015-06-08 01:38:54              16   
2     1359  2015-01-01 18:52:44  2015-01-01 18:52:45              15   
3   150084  2015-04-28 21:13:25  2015-05-04 13:54:50              44   
4   221365  2015-07-21 07:09:52  2015-09-09 18:40:53              39   

   device_id  source  browser  sex  age    ip_address  class  
0      89215       2        0    1   39  7.327584e+08      0  
1      24078       0        0    0   53  3.503114e+08      0  
2     131216       2        3    1   53  2.621474e+09      1  
3       3977       2        4    1   41  3.840542e+09      0  
4      68757       0        4    1   45  4.155831e+08      0  


In [24]:
# Feature and Target Separation
X_fraud = fraud_data.drop('class', axis=1)
y_fraud = fraud_data['class']

# Train-Test Split for Fraud Data dataset
from sklearn.model_selection import train_test_split

X_train_fraud, X_test_fraud, y_train_fraud, y_test_fraud = train_test_split(X_fraud, y_fraud, test_size=0.3, random_state=42)


In [29]:
# Drop datetime columns before scaling
X_train_fraud = X_train_fraud.drop(columns=['signup_time', 'purchase_time'])
X_test_fraud = X_test_fraud.drop(columns=['signup_time', 'purchase_time'])

from sklearn.preprocessing import StandardScaler

# Define the scaler
scaler = StandardScaler()

# Scale the features for Fraud Data dataset
X_train_fraud_scaled = scaler.fit_transform(X_train_fraud)
X_test_fraud_scaled = scaler.transform(X_test_fraud)


In [30]:
from sklearn.preprocessing import StandardScaler

# Define the scaler
scaler = StandardScaler()

# Scale the features for Fraud Data dataset
X_train_fraud_scaled = scaler.fit_transform(X_train_fraud)
X_test_fraud_scaled = scaler.transform(X_test_fraud)


In [31]:
# Save the cleaned fraud data to a CSV file
fraud_data.to_csv('cleaned_fraud_data.csv', index=False)

# Verify the file is saved correctly
print("Cleaned fraud data saved as 'cleaned_fraud_data.csv'")


Cleaned fraud data saved as 'cleaned_fraud_data.csv'


In [32]:
import shap
from sklearn.ensemble import RandomForestClassifier

# Define the Random Forest model
rf_fraud = RandomForestClassifier()

# Train the model
rf_fraud.fit(X_train_fraud_scaled, y_train_fraud)


RandomForestClassifier()

In [ ]:
import shap
import pandas as pd

# Sample a fraction of the test data
sampled_data = X_test_fraud_scaled.sample(frac=0.1, random_state=42)

# Calculate SHAP values for the sampled data
explainer = shap.TreeExplainer(rf_fraud)
shap_values = explainer.shap_values(sampled_data)